In [3]:
import pandas as pd

# State abbreviation mapping
state_abbreviation_mapping = {
    "utah": "ut",
    "finland": "NA",
    "georgia": "ga",
    "israel": "NA",
    "texas": "tx",
    "taiwan": "NA",
    "nigeria": "NA",
    "ohio": "oh",
    "mali": "NA",
    "colorado": "co",
    "wyoming": "wy",
    "minnesota": "mn"
}

# Load the CSV file
input_file = "Utes_Mens.csv"  # Replace with your file path if needed
player_data = pd.read_csv(input_file)

# Create the "School" column (first word of "High School")
player_data["School"] = player_data["High School"].apply(lambda x: str(x).split()[0])

# Create the "State" column (state abbreviation from "Hometown")
player_data["State"] = player_data["Hometown"].apply(
    lambda x: state_abbreviation_mapping.get(x.split(",")[-1].strip().lower(), "NA")
)

# Save the updated DataFrame to a new CSV file
output_file = "Updated_Player_Data.csv"
player_data.to_csv(output_file, index=False)

print(f"Updated data saved to {output_file}")


Updated data saved to Updated_Player_Data.csv


In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

# Function to get player profiles
def get_player_profile(base_search_url, player_name, state, sport, school_keyword):
    search_url = f"{base_search_url}?q={player_name.replace(' ', '+')}&state={state}&sport={sport}"
    print(f"Searching for player: {search_url}")

    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"Failed to load search page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    player_links = soup.find_all('a', href=True)

    profiles = []
    for link in player_links:
        href = link.get("href", "")
        if "/athletes/" in href and school_keyword.lower() in href.lower():
            profiles.append("https://www.maxpreps.com" + href)

    return profiles if profiles else None

# Function to find a valid basketball stats URL
def find_valid_stats_url(profiles, sport="basketball"):
    for profile_url in profiles:
        response = requests.get(profile_url)
        if response.status_code != 200:
            print(f"Failed to load player profile: {profile_url}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        stats_link = soup.find('a', string='Stats')

        if stats_link:
            stats_url = "https://www.maxpreps.com" + stats_link['href']
            if sport in stats_url:
                return stats_url

    print("No valid stats URL found for basketball.")
    return None

# Function to fetch combined stats
def fetch_combined_stats(stats_url):
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(stats_url)
        time.sleep(5)

        # General stats
        rows = driver.find_elements(By.XPATH, '//tbody[contains(@class, "table-body")]/tr')
        general_stats = {}
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, 'td')
            if len(cells) > 8:
                general_stats = {
                    'GP': int(cells[0].text.strip()),
                    'PPG': cells[2].text.strip(),
                    'RPG': cells[5].text.strip(),
                    'APG': cells[6].text.strip(),
                    'SPG': cells[7].text.strip(),
                    'BPG': cells[8].text.strip(),
                }
                break

        # Shooting stats (1)
        shooting_tab = driver.find_element(By.XPATH, '//button[@title="Shooting"]')
        shooting_tab.click()
        time.sleep(3)

        shooting_1_section = driver.find_element(By.XPATH, '//h2[text()="Shooting (1)"]/following-sibling::div')
        rows_1 = shooting_1_section.find_elements(By.XPATH, './/tbody/tr')
        shooting_1_stats = {}
        for row in rows_1:
            cells = row.find_elements(By.TAG_NAME, 'td')
            if len(cells) > 5:
                shooting_1_stats = {
                    'PTS': cells[2].text.strip(),
                    'FGM': cells[3].text.strip(),
                    'FGA': cells[4].text.strip(),
                    'FG%': cells[5].text.strip(),
                }
                break

        # Shooting stats (2)
        shooting_2_section = driver.find_element(By.XPATH, '//h2[text()="Shooting (2)"]/following-sibling::div')
        rows_2 = shooting_2_section.find_elements(By.XPATH, './/tbody/tr')
        shooting_2_stats = {}
        for row in rows_2:
            cells = row.find_elements(By.TAG_NAME, 'td')
            if len(cells) > 8:
                shooting_2_stats = {
                    '3PA': cells[3].text.strip(),
                    '3PM': cells[4].text.strip(),
                    '3P%': cells[5].text.strip(),
                }
                break

        combined_stats = {**general_stats, **shooting_1_stats, **shooting_2_stats}
        return combined_stats if combined_stats else "No combined stats found."

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    finally:
        driver.quit()

# Load player data from CSV
file_path = "Updated_Player_Data.csv"
player_data = pd.read_csv(file_path)

# Base search URL for MaxPreps
base_search_url = "https://www.maxpreps.com/search/"

# Loop through each player
for _, row in player_data.iterrows():
    player_name = row["Name"]
    state = row["State"]
    school_keyword = row["School"]

    if state == "NA" or not school_keyword:
        print(f"Skipping {player_name} due to missing state or school keyword.")
        continue

    print(f"Processing Player: {player_name}, State: {state}, School Keyword: {school_keyword}")

    # Step 1: Get player profiles
    player_profiles = get_player_profile(base_search_url, player_name, state, "basketball", school_keyword)
    if player_profiles:
        print(f"Found Player Profiles: {player_profiles}")

        # Step 2: Find a valid stats URL
        stats_url = find_valid_stats_url(player_profiles, "basketball")
        if stats_url:
            print(f"Stats URL Found: {stats_url}")

            # Step 3: Fetch stats
            combined_stats = fetch_combined_stats(stats_url)
            if combined_stats:
                print(f"Combined Stats for {player_name}: {combined_stats}")
            else:
                print(f"No stats found for {player_name}.")
        else:
            print(f"No valid stats URL for {player_name}.")
    else:
        print(f"No profiles found for {player_name}.")


Processing Player: Hunter Erickson, State: ut, School Keyword: Timpview
Searching for player: https://www.maxpreps.com/search/?q=Hunter+Erickson&state=ut&sport=basketball
Found Player Profiles: ['https://www.maxpreps.com/ut/provo/timpview-thunderbirds/athletes/hunter-erickson/?careerid=3q91prrpb1sff', 'https://www.maxpreps.com/ut/provo/timpview-thunderbirds/athletes/hunter-ericson/?careerid=en85jc79auosc']
No valid stats URL found for basketball.
No valid stats URL for Hunter Erickson.
Processing Player: Miro Little, State: nan, School Keyword: Sunrise
Searching for player: https://www.maxpreps.com/search/?q=Miro+Little&state=nan&sport=basketball
No profiles found for Miro Little.
Processing Player: Ezra Ausar, State: nan, School Keyword: Liberty
Searching for player: https://www.maxpreps.com/search/?q=Ezra+Ausar&state=nan&sport=basketball
No profiles found for Ezra Ausar.
Processing Player: Joul Karram, State: nan, School Keyword: Hakfar
Searching for player: https://www.maxpreps.com/

No stats found for Mike Sharavjamts.
Processing Player: Ibi Traore, State: nan, School Keyword: West
Searching for player: https://www.maxpreps.com/search/?q=Ibi+Traore&state=nan&sport=basketball
No profiles found for Ibi Traore.
Processing Player: Zach Keller, State: co, School Keyword: ThunderRidge
Searching for player: https://www.maxpreps.com/search/?q=Zach+Keller&state=co&sport=basketball
Found Player Profiles: ['https://www.maxpreps.com/co/highlands-ranch/thunderridge-grizzlies/athletes/zach-keller/?careerid=3b4h6c0jpi0g0']
Stats URL Found: https://www.maxpreps.com/co/highlands-ranch/thunderridge-grizzlies/athletes/zach-keller/basketball/stats/?careerid=3b4h6c0jpi0g0
An error occurred: Message: element click intercepted: Element is not clickable at point (285, 870)
  (Session info: chrome=133.0.6943.53)
Stacktrace:
	GetHandleVerifier [0x00007FF619CC2EC5+28789]
	(No symbol) [0x00007FF619C2F870]
	(No symbol) [0x00007FF619AC8F9A]
	(No symbol) [0x00007FF619B270C9]
	(No symbol) [0x000

In [ ]:
# For each csv files

In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time


def get_player_profile(base_search_url, player_name, state, sport, school_keyword):
    """Searches for a player's profile link on MaxPreps and filters the correct one based on the school keyword."""
    search_url = f"{base_search_url}?q={player_name.replace(' ', '+')}&state={state}&sport={sport}"
    print(f"Searching for player: {search_url}")

    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"Failed to load search page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    player_links = soup.find_all('a', href=True)

    profiles = []
    for link in player_links:
        href = link.get("href", "")
        if "/athletes/" in href and school_keyword.lower() in href.lower():
            profiles.append("https://www.maxpreps.com" + href)

    return profiles if profiles else None


def find_valid_stats_url(profiles, sport="basketball"):
    """Finds the stats URL for basketball by checking each profile."""
    for profile_url in profiles:
        response = requests.get(profile_url)
        if response.status_code != 200:
            print(f"Failed to load player profile: {profile_url}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        stats_link = soup.find('a', string='Stats')

        if stats_link:
            stats_url = "https://www.maxpreps.com" + stats_link['href']
            if sport in stats_url:
                return stats_url  # Return the first valid basketball stats URL

    print("No valid stats URL found for basketball.")
    return None


def fetch_combined_stats(stats_url):
    """Fetch combined stats including general and shooting stats."""
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(stats_url)
        time.sleep(5)

        # General stats (GP, PPG, RPG, APG, SPG, BPG)
        rows = driver.find_elements(By.XPATH, '//tbody[contains(@class, "table-body")]/tr')
        general_stats = {}
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, 'td')
            if len(cells) > 8:
                general_stats = {
                    'GP': int(cells[0].text.strip()),
                    'PPG': cells[2].text.strip(),
                    'RPG': cells[5].text.strip(),
                    'APG': cells[6].text.strip(),
                    'SPG': cells[7].text.strip(),
                    'BPG': cells[8].text.strip(),
                }
                break

        # Click on the "Shooting" tab
        shooting_tab = driver.find_element(By.XPATH, '//button[@title="Shooting"]')
        shooting_tab.click()
        time.sleep(3)

        # Shooting stats (1)
        shooting_1_section = driver.find_element(By.XPATH, '//h2[text()="Shooting (1)"]/following-sibling::div')
        rows_1 = shooting_1_section.find_elements(By.XPATH, './/tbody/tr')
        shooting_1_stats = {}
        for row in rows_1:
            cells = row.find_elements(By.TAG_NAME, 'td')
            if len(cells) > 5:
                shooting_1_stats = {
                    'PTS': cells[2].text.strip(),
                    'FGM': cells[3].text.strip(),
                    'FGA': cells[4].text.strip(),
                    'FG%': cells[5].text.strip(),
                }
                break

        # Shooting stats (2)
        shooting_2_section = driver.find_element(By.XPATH, '//h2[text()="Shooting (2)"]/following-sibling::div')
        rows_2 = shooting_2_section.find_elements(By.XPATH, './/tbody/tr')
        shooting_2_stats = {}
        for row in rows_2:
            cells = row.find_elements(By.TAG_NAME, 'td')
            if len(cells) > 8:
                shooting_2_stats = {
                    '3PA': cells[3].text.strip(),
                    '3PM': cells[4].text.strip(),
                    '3P%': cells[5].text.strip(),
                }
                break

        # Combine all stats
        combined_stats = {**general_stats, **shooting_1_stats, **shooting_2_stats}
        return combined_stats if combined_stats else "No combined stats found."

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    finally:
        driver.quit()


# Load cleaned player data
player_data = pd.read_csv("Snow_Badgers_Womens.csv")

# Create an empty DataFrame to store players with stats
players_with_stats = pd.DataFrame(columns=player_data.columns.tolist() + ["GP", "PPG", "RPG", "APG", "SPG", "BPG", "PTS", "FGM", "FGA", "FG%", "3PA", "3PM", "3P%"])

# Iterate through each player and fetch stats
base_search_url = "https://www.maxpreps.com/search/"
for index, row in player_data.iterrows():
    player_name = row["Name"]
    state = row["State"]
    school_keyword = row["School"]

    print(f"Processing Player: {player_name}, State: {state}, School Keyword: {school_keyword}")

    # Step 1: Get player profile URLs
    player_profiles = get_player_profile(base_search_url, player_name, state, "basketball", school_keyword)

    if player_profiles:
        print(f"Found Player Profiles: {player_profiles}")

        # Step 2: Find the valid stats URL for basketball
        stats_url = find_valid_stats_url(player_profiles, "basketball")
        if stats_url:
            print(f"Stats URL Found: {stats_url}")

            # Step 3: Fetch combined stats
            combined_stats = fetch_combined_stats(stats_url)
            if combined_stats:
                # Append player stats to the DataFrame using pd.concat()
                player_stats = pd.DataFrame([{**row.to_dict(), **combined_stats}])
                players_with_stats = pd.concat([players_with_stats, player_stats], ignore_index=True)
        else:
            print("No valid stats URL found.")
    else:
        print("Player not found.")

# Display the resulting DataFrame
print(players_with_stats)

# Save to CSV
players_with_stats.to_csv("Snow_Badgers_Womens_Stats.csv", index=False)


Processing Player: Hadley Richins, State: ut, School Keyword: north
Searching for player: https://www.maxpreps.com/search/?q=Hadley+Richins&state=ut&sport=basketball
Found Player Profiles: ['https://www.maxpreps.com/ut/coalville/north-summit-braves/athletes/hadley-richins/?careerid=fjum2fhpg1988', 'https://www.maxpreps.com/ut/coalville/north-summit-braves/athletes/hailey-richins/?careerid=3s0jrib2h7a44']
Stats URL Found: https://www.maxpreps.com/ut/coalville/north-summit-braves/athletes/hadley-richins/basketball/girls/stats/?careerid=fjum2fhpg1988
Processing Player: Brooke Pennington, State: ut, School Keyword: Springville
Searching for player: https://www.maxpreps.com/search/?q=Brooke+Pennington&state=ut&sport=basketball
Found Player Profiles: ['https://www.maxpreps.com/ut/springville/springville-red-devils/athletes/brooke-pennington/?careerid=uru1evova59dd']
Stats URL Found: https://www.maxpreps.com/ut/springville/springville-red-devils/athletes/brooke-pennington/basketball/girls/sta

# Combining All FOR MEN

In [5]:
import pandas as pd
import os

# Define the list of filenames
file_names = [
    "BYU_Mens_Stats.csv",
    "Snow_Badgers_Mens_Stats.csv",
    "Weber_State_Mens_Stats.csv",
    "UTAH_VALLEY_Mens_Stats.csv",
    "Southern_Utah_Mens_Stats.csv",
    "Utah_Tech_Mens_Stats.csv",
    "Westminster_Mens_Stats.csv",
    "Salt_Lake_Community_Mens_Stats.csv",
    "Utah_State_Eagles_Mens_Stats.csv",
    "Utes_Mens_Stats.csv",
    "Utah_State_Mens_Stats.csv"
]

# Initialize an empty list to store DataFrames
dataframes = []

# Load each file and add the college name column
for file in file_names:
    try:
        # Load the CSV file
        df = pd.read_csv(file)
        
        # Extract the college name from the file name (removing _Mens_Stats.csv)
        college_name = file.replace("_Mens_Stats.csv", "").replace("_", " ")
        
        # Add a new column with the college name
        df["College"] = college_name
        
        # Append the DataFrame to the list
        dataframes.append(df)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Combine all DataFrames into a single DataFrame
combined_data = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_file_name = "Combined_Mens_Basketball_Stats.csv"
combined_data.to_csv(combined_file_name, index=False)

print(f"Combined dataset saved as {combined_file_name}")


Combined dataset saved as Combined_Mens_Basketball_Stats.csv


In [7]:
import pandas as pd

# Define the list of filenames
file_names = [
    "BYU_mens_Stats.csv",
    "Snow_Badgers_Mens_Stats.csv",
    "Weber_State_Mens_Stats.csv",
    "UTAH_VALLEY_Mens_Stats.csv",
    "Southern_Utah_Mens_Stats.csv",
    "Utah_Tech_Mens_Stats.csv",
    "Westminster_Mens_Stats.csv",
    "Salt_Lake_Community_Mens_Stats.csv",
    "Utah_State_Eagles_Mens_Stats.csv",
    "Utes_mens_Stats.csv",
    "Utah_State_Mens_Stats.csv"
]

# Initialize an empty list to store DataFrames
dataframes = []
row_counts = {}  # Dictionary to store row counts of each file

# Load each file and add the college name column
for file in file_names:
    try:
        # Load the CSV file
        df = pd.read_csv(file)
        
        # Count the number of rows
        row_counts[file] = len(df)
        
        # Extract the college name from the file name (removing _Mens_Stats.csv)
        college_name = file.replace("_Mens_Stats.csv", "").replace("_", " ")
        
        # Add a new column with the college name
        df["College"] = college_name
        
        # Append the DataFrame to the list
        dataframes.append(df)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Calculate the total number of rows before merging
total_rows_before = sum(row_counts.values())

# Combine all DataFrames into a single DataFrame
combined_data = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_file_name = "Combined_Mens_Basketball_Stats.csv"
combined_data.to_csv(combined_file_name, index=False)

# Get the total rows in the final dataset
total_rows_after = len(combined_data)

# Display results for verification
print("Row counts for each file:")
for file, count in row_counts.items():
    print(f"{file}: {count} rows")

print(f"\nTotal rows before merging: {total_rows_before}")
print(f"Total rows in the final combined dataset: {total_rows_after}")

# Cross-verification check
if total_rows_before == total_rows_after:
    print("\n✅ Row count verification PASSED: The number of rows match.")
else:
    print("\n❌ Row count verification FAILED: Mismatch in row counts!")


Row counts for each file:
BYU_mens_Stats.csv: 12 rows
Snow_Badgers_Mens_Stats.csv: 12 rows
Weber_State_Mens_Stats.csv: 8 rows
UTAH_VALLEY_Mens_Stats.csv: 12 rows
Southern_Utah_Mens_Stats.csv: 13 rows
Utah_Tech_Mens_Stats.csv: 9 rows
Westminster_Mens_Stats.csv: 10 rows
Salt_Lake_Community_Mens_Stats.csv: 8 rows
Utah_State_Eagles_Mens_Stats.csv: 10 rows
Utes_mens_Stats.csv: 8 rows
Utah_State_Mens_Stats.csv: 7 rows

Total rows before merging: 109
Total rows in the final combined dataset: 109

✅ Row count verification PASSED: The number of rows match.


# COMBINING ALL FOR WOMEN

In [13]:
import pandas as pd

# Define the list of filenames
file_names = [
    "BYU_Womens_Stats.csv",
    "Snow_Badgers_Womens_Stats.csv",
    "Weber_State_Womens_Stats.csv",
    "UTAH_VALLEY_Womens_Stats.csv",
    "Southern_Utah_Womens_Stats.csv",
    "Utah_Tech_Womens_Stats.csv",
    "Westminster_Womens_Stats.csv",
    "Salt_Lake_Community_Womens_Stats.csv",
    "Utah_State_Eagles_Womens_Stats.csv",
    "Utes_Womens_Stats.csv",
    "Utah_State_Womens_Stats.csv"
]

# Initialize an empty list to store DataFrames
dataframes = []
row_counts = {}  # Dictionary to store row counts of each file

# Load each file and add the college name column
for file in file_names:
    try:
        # Load the CSV file
        df = pd.read_csv(file)
        
        # Count the number of rows
        row_counts[file] = len(df)
        
        # Extract the college name from the file name (removing _Mens_Stats.csv)
        college_name = file.replace("_Womens_Stats.csv", "").replace("_", " ")
        
        # Add a new column with the college name
        df["College"] = college_name
        
        # Append the DataFrame to the list
        dataframes.append(df)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Calculate the total number of rows before merging
total_rows_before = sum(row_counts.values())

# Combine all DataFrames into a single DataFrame
combined_data = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_file_name = "Combined_Womens_Basketball_Stats.csv"
combined_data.to_csv(combined_file_name, index=False)

# Get the total rows in the final dataset
total_rows_after = len(combined_data)

# Display results for verification
print("Row counts for each file:")
for file, count in row_counts.items():
    print(f"{file}: {count} rows")

print(f"\nTotal rows before merging: {total_rows_before}")
print(f"Total rows in the final combined dataset: {total_rows_after}")

# Cross-verification check
if total_rows_before == total_rows_after:
    print("\n✅ Row count verification PASSED: The number of rows match.")
else:
    print("\n❌ Row count verification FAILED: Mismatch in row counts!")


Row counts for each file:
BYU_Womens_Stats.csv: 9 rows
Snow_Badgers_Womens_Stats.csv: 12 rows
Weber_State_Womens_Stats.csv: 11 rows
UTAH_VALLEY_Womens_Stats.csv: 12 rows
Southern_Utah_Womens_Stats.csv: 6 rows
Utah_Tech_Womens_Stats.csv: 11 rows
Westminster_Womens_Stats.csv: 9 rows
Salt_Lake_Community_Womens_Stats.csv: 7 rows
Utah_State_Eagles_Womens_Stats.csv: 12 rows
Utes_Womens_Stats.csv: 5 rows
Utah_State_Womens_Stats.csv: 8 rows

Total rows before merging: 102
Total rows in the final combined dataset: 102

✅ Row count verification PASSED: The number of rows match.


In [19]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("Combined_Womens_Basketball_Stats.csv")  # Replace with your actual file path

# Get the number of rows
num_rows = len(df)

# Print the number of rows
print(f"Number of rows: {num_rows}")

# Print all rows
print(df)


Number of rows: 102
        Season     College              Name  Height       Position  \
0    2024-2025         BYU     Naia Tanuvasa   5'10"          Guard   
1    2024-2025         BYU     Amari Whiting   5'10"          Guard   
2    2024-2025         BYU       Kylie Krebs    5'7"          Guard   
3    2024-2025         BYU    Kambree Barber    6'0"          Guard   
4    2024-2025         BYU    Heather Hamson    6'3"        Forward   
..         ...         ...               ...     ...            ...   
97     2024-25  Utah State  Elise Livingston   5' 9"          Guard   
98     2024-25  Utah State   Taliyah Logwood   5' 9"  Guard/Forward   
99     2024-25  Utah State        Mia Tarver   5' 7"          Guard   
100    2024-25  Utah State   Cheyenne Stubbs   5' 5"          Guard   
101    2024-25  Utah State    Jamisyn Heaton  5' 10"          Guard   

    Academic Year  GP   PPG   RPG  APG  SPG  BPG  PTS  FGM  FGA  FG%  3PM  \
0        Freshman  25  14.5   5.4  3.2  3.0  0.7  

In [20]:
null_counts = df.isnull().sum()

# Print the number of null values per column
print("Number of null values per column:\n", null_counts)

# Get the total number of null values in the entire dataset
total_nulls = df.isnull().sum().sum()
print(f"Total number of null values in the dataset: {total_nulls}")

Number of null values per column:
 Season           0
College          0
Name             0
Height           0
Position         0
Academic Year    0
GP               0
PPG              0
RPG              0
APG              0
SPG              0
BPG              0
PTS              0
FGM              0
FGA              0
FG%              0
3PM              0
3PA              0
3P%              0
dtype: int64
Total number of null values in the dataset: 0
